In [ ]:
!pip install ipython-autotime
%load_ext autotime

!apt-get install openjdk-8-jdk-headless -qq > /dev/null

!wget -q https://dlcdn.apache.org/spark/spark-3.0.3/spark-3.0.3-bin-hadoop2.7.tgz

!tar xf /content/spark-3.0.3-bin-hadoop2.7.tgz

!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.3-bin-hadoop2.7"

import findspark
findspark.init()


from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local")\
        .appName("MovieSort")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [ ]:
#unrar ratings and movies csv
!unrar e /content/drive/MyDrive/ratings.rar
!unrar e /content/drive/MyDrive/movies.rar

In [ ]:
#read csv files as spark dataframe
ratingsDf = spark.read.csv("/content/ratings.csv", inferSchema=True, header = True)
moviesDf = spark.read.csv("/content/movies.csv", inferSchema=True, header = True)

In [ ]:
#join tables movies and ratings
joinedDf =ratingsDf.alias("ratings").join(moviesDf.alias("movies"),ratings.movieId == movies.movieId,"inner").select("ratings.userId","ratings.movieId","movies.title","ratings.rating","movies.genres")
joinedDf.show()


In [ ]:
#group dataframe by title
groupTitle = joinedDf.groupBy("title").agg(sum('rating').alias("sum_rating"), \
                                           count("rating").alias("count_rating"),\
                                           avg("rating").alias("average_rating"),first("genres").alias("genres"))\
                                           .sort(desc("sum_rating"))
groupTitle.show()

In [ ]:
#sort movies by sum_rating to show top10
groupTitle_sorted = groupTitle.sort(desc("sum_rating"),desc("average_rating"))

In [ ]:
#write dataframe into csv
groupTitle_sorted.write.option("header",True).csv("/content/drive/MyDrive/data_images/sortedmovies.csv")

In [ ]:
#concatenate files into csv
os.system("cat /content/drive/MyDrive/data_images/sortedmovies.csv/p* > /content/drive/MyDrive/data_images/sortedmovies.csv")